# Hierarchical Training of inception V3 model on aerial data
instead of training a model to classify images into species directly, we will train a model to first classify them into leaf type, then two models which classify broadleaf and pineneedle trees into their respective families. This should hopefully result in a more general and more accurate classification

### Loading in the aerial hierarchy dictionary and image data

In [1]:
import numpy as np
import json

with open(r"..\..\data\general_hierarchy.json") as file:
    general_hierarchy = json.load(file)

In [2]:
#loading image data
train_images = np.load(r"C:\Users\bench\OneDrive\Documents\EMAT Year 3\MDM3\Phase C\ratio_adjusted_aerial_dataset\train_images.npy")
train_labels = np.load(r"C:\Users\bench\OneDrive\Documents\EMAT Year 3\MDM3\Phase C\ratio_adjusted_aerial_dataset\train_labels.npy")
val_images = np.load(r"C:\Users\bench\OneDrive\Documents\EMAT Year 3\MDM3\Phase C\ratio_adjusted_aerial_dataset\val_images.npy")
val_labels = np.load(r"C:\Users\bench\OneDrive\Documents\EMAT Year 3\MDM3\Phase C\ratio_adjusted_aerial_dataset\val_labels.npy")

# Training the initial binary classifier

### Important training parameters

In [3]:
import torch
num_classes = 2
feature_extract = True

### creating binary labels

In [5]:
#getting two lists of species for binary classification
binary_species = {"broadleaf":[], "conifer":[]}
for leaf_type, foliage in zip(["broadleaf","conifer"], general_hierarchy):
    for family in general_hierarchy[foliage]:
        for species in general_hierarchy[foliage][family]:
            binary_species[leaf_type].append(species)

print(binary_species)

# converting image labels to binary labels, 0 represents broadleaf and 1 represents conifer
def getFoliageLabels(aerial_labels, binary_species):
    foliage_labels = np.empty((aerial_labels.shape[0],))
    foliage_labels.fill(np.nan)
    broadleaf_index = np.isin(aerial_labels, binary_species["broadleaf"])
    conifer_index = np.isin(aerial_labels, binary_species["conifer"])
    
    foliage_labels[broadleaf_index] = 0
    foliage_labels[conifer_index] = 1
    foliage_labels = foliage_labels.astype("uint8")
    
    #check if any labels are not broadleaf or conifer
    if np.count_nonzero(foliage_labels==np.nan) != 0:
        raise ValueError("Some labels are not broadleaf or conifer")
    
    return foliage_labels

foliage_train_labels = getFoliageLabels(train_labels, binary_species)
foliage_val_labels = getFoliageLabels(val_labels, binary_species)

#displaying some random labels
print(foliage_train_labels[np.random.randint(0, foliage_train_labels.shape[0], 100)])

{'broadleaf': ['Quercus_rubra', 'Quercus_robur', 'Quercus_petraea', 'Fagus_sylvatica', 'Acer_pseudoplatanus', 'Fraxinus_excelsior', 'Tilia_spec.', 'Prunus_spec.', 'Alnus_spec.', 'Populus_spec.', 'Betula_spec.'], 'conifer': ['Picea_abies', 'Abies_alba', 'Pseudotsuga_menziesii', 'Pinus_sylvestris', 'Pinus_nigra', 'Pinus_strobus', 'Larix_decidua', 'Larix_kaempferi']}
[1 1 0 0 1 0 1 1 0 1 0 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1
 0 1 0 1 1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 0 0 1 1 0 1 1]


### verifying data dimensions are as expected

In [6]:
print("train labels: ",foliage_train_labels.shape)
print("val labels: ",foliage_val_labels.shape)
print("train images: ",train_images.shape)
print("val images: ",val_images.shape)

train labels:  (14094,)
val labels:  (1762,)
train images:  (14094, 304, 304, 4)
val images:  (1762, 304, 304, 4)


### initialising model

In [7]:
from training_funcs import initialize_inception_model

foliage_model, input_size = initialize_inception_model(num_classes, feature_extract, use_pretrained=True)

print(input_size)

C:\Users\bench\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in C:\Users\bench/.cache\torch\hub\pytorch_vision_v0.10.0


299


### Creating data loaders

In [8]:
from training_funcs import CustomDataset
    
#creating training and validation pytorch datasets
training_dataset = CustomDataset(train_images, foliage_train_labels, input_size)
val_dataset = CustomDataset(val_images, foliage_val_labels, input_size)
batch_size = 32
# Create training and validation dataloaders
dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True),
                    'val': torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)}

### Training the binary model

In [10]:
#checking the params we are training
params_to_update = []
names = []
for name,param in foliage_model.named_parameters():
  if param.requires_grad==True:
    params_to_update.append(param)
    names.append(name)

print("number of parameters to train =",len(params_to_update))
for i, param in enumerate(params_to_update):
  print("parameter {}:".format(names[i]),param.shape)

number of parameters to train = 6
parameter pre_layer.input_conv.weight: torch.Size([3, 4, 1, 1])
parameter pre_layer.input_conv.bias: torch.Size([3])
parameter inception_v3.AuxLogits.fc.weight: torch.Size([2, 768])
parameter inception_v3.AuxLogits.fc.bias: torch.Size([2])
parameter inception_v3.fc.weight: torch.Size([2, 2048])
parameter inception_v3.fc.bias: torch.Size([2])


In [11]:
from torch import nn
from torch import optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("sending model to",device)
# Send the model to GPU
foliage_model = foliage_model.to(device)
# Setup the loss function
criterion = nn.CrossEntropyLoss()
#initializing optimizer with hyperparameters determined from the random search
optimizer = optim.AdamW(params_to_update, lr=0.0005, betas=(0.9, 0.995), weight_decay=0.015)


sending model to cuda:0


In [12]:
from training_funcs import train_model
import os
import torch
#fixing random seed for consistent results
torch.manual_seed(0)

#setting the number of epochs
num_epochs = 300
# Train and evaluate
foliage_model, convergence_dict = train_model(foliage_model,
                                                dataloaders_dict,
                                                criterion,
                                                optimizer,
                                                num_epochs=num_epochs,
                                                is_inception=True,
                                                tensorboard_writer=None,
                                                early_stopping=False,
                                                device=device)

#saving the model
filename = "foliage_model.pth"
if not os.path.exists(filename) or filename == "temp.pth":
    torch.save(foliage_model.state_dict(),'foliage_model.pth')
else:
    print("model file already exists")

Epoch 0/299
----------
train Loss: 0.5901 Acc: 0.7895
val Loss: 0.3429 Acc: 0.8632
epoch 0 complete
Epoch 1/299
----------
train Loss: 0.4872 Acc: 0.8427
val Loss: 0.3154 Acc: 0.8655
epoch 1 complete
Epoch 2/299
----------
train Loss: 0.4732 Acc: 0.8443
val Loss: 0.3398 Acc: 0.8570
epoch 2 complete
Epoch 3/299
----------
train Loss: 0.4769 Acc: 0.8472
val Loss: 0.3007 Acc: 0.8700
epoch 3 complete
Epoch 4/299
----------
train Loss: 0.4594 Acc: 0.8521
val Loss: 0.2952 Acc: 0.8740
epoch 4 complete
Epoch 5/299
----------
train Loss: 0.4568 Acc: 0.8529
val Loss: 0.2858 Acc: 0.8854
epoch 5 complete
Epoch 6/299
----------
train Loss: 0.4551 Acc: 0.8541
val Loss: 0.2833 Acc: 0.8820
epoch 6 complete
Epoch 7/299
----------
train Loss: 0.4599 Acc: 0.8527
val Loss: 0.2728 Acc: 0.8871
epoch 7 complete
Epoch 8/299
----------
train Loss: 0.4432 Acc: 0.8546
val Loss: 0.2702 Acc: 0.8899
epoch 8 complete
Epoch 9/299
----------
train Loss: 0.4456 Acc: 0.8546
val Loss: 0.2775 Acc: 0.8820
epoch 9 complete


In [13]:
#saving the convergence dictionary
with open("foliage_convergence.json", "w") as file:
    json.dump(convergence_dict, file)

# Training the conifer family classifier

In [ ]:
conifer_dict = general_hierarchy["conifer"]

#getting images corresponding to conifers
conifer_species = []
for i, family in enumerate(conifer_dict):
    for species in conifer_dict[family]:
        print(species)
        conifer_species.append(species)

# converting species labels, to numeric family labels
def getFamilyLabels(aerial_labels, species_list):
    foliage_labels = np.empty((aerial_labels.shape[0],))
    foliage_labels.fill(np.nan)
    broadleaf_index = np.isin(aerial_labels, binary_species["broadleaf"])
    conifer_index = np.isin(aerial_labels, binary_species["conifer"])
    
    foliage_labels[broadleaf_index] = 0
    foliage_labels[conifer_index] = 1
    foliage_labels = foliage_labels.astype("uint8")
    
    #check if any labels are not broadleaf or conifer
    if np.count_nonzero(foliage_labels==np.nan) != 0:
        raise ValueError("Some labels are not broadleaf or conifer")
    
    return foliage_labels

In [ ]:
print(family_list)
print(np.unique(conifer_labels))
print(conifer_labels.shape)
print(conifer_images.shape)
print(conifer_labels[0:1000])

#verifying that there are no zeros contained in the final list of labels
if np.count_nonzero(conifer_labels == 0) != 0:
    print(np.where(conifer_labels == 0))
    raise ValueError("unlabelled data present")